# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [ ]:
%pip install readability-lxml
# доп. часть сделана через этот модуль, интересно было посмотреть его

In [23]:
import requests
from bs4 import BeautifulSoup
import sys
import re
import pandas as pd
from readability import Document

def log_exception(msg, exc = None):    
    if exc is None:
        exc = sys.exc_info()[0]
    print(f"Caught exception: {exc}")
    print(f"Error: {msg}")
    

def fetch_page(url):
    resp  = requests.get(url)
    if resp.status_code == 200:
        return resp.text
    raise Exception(f'Bad response code: {resp.status_code}')
    
def parse_post(post):
    res =  None
    try:
        e_user  = post.select('a.post__user-info')[0]
        e_title = post.select('.post__title_link')[0]
        e_text  = post.select('.post__body > div.post__text')[0]
        res     = {'time': post.select('span.post__time')[0].text, # допустим она "распарсена"
                   'title': e_title.text,
                   'url': e_title['href'],
                   # доп поля
                   'author_name': e_user.text,
                   'author_url': e_user['href'],
                   'preview': e_text.text,
                    
                   }
        for k in res:
            res[k] = res[k].strip()
    except:        
        post_text = re.sub(r'\s+', ' ', post.text[0:128])
        log_exception(f'Failed to parse post: {post_text}')
    return res

def fetch_post_text(post_url):
    try:
        html = fetch_page(post_url)
        return BeautifulSoup(Document(html).summary(), 'html.parser').text
    except:
        log_exception(f'Failed to fetch post: {post_url}')
    return None

def parse_posts_previews(html):
    page = BeautifulSoup(html, 'html.parser')
    post_selector = 'article.post_preview'
    res = []
    for post in page.select(post_selector):
        res.append(parse_post(post))
    return res
    
    
html = fetch_page('https://habr.com/ru/all')
with open('/tmp/h.html', 'w') as f:
    f.write(html)
posts = parse_posts_previews(html)

KEYWORDS = ['python', 'парсинг', 'go']
kw_re = re.compile('|'.join(KEYWORDS))

filtered_posts = [p for p in posts if kw_re.search(p['preview'].lower()+p['title'].lower())]

df = pd.DataFrame(filtered_posts)
df['text'] = df['url'].apply(lambda post_url: fetch_post_text(post_url))
display(df)

print("First text example: ")
print(df.iloc[0]['text'])


,time,title,url,author_name,author_url,preview,text
0,сегодня в 18:34,Как в 2020 году разработчику найти удаленную р...,https://habr.com/ru/company/gms/blog/515200/,alexlash,https://habr.com/ru/users/alexlash/,"По статистике Stack Overflow, активно ищут раб...","\n\r\nПо статистике Stack Overflow, активно ищ..."


First text example: 


По статистике Stack Overflow, активно ищут работу немногим более 17% разработчиков. При этом, помимо зарплаты, в пятерку важных факторов при смене места входит и запрос на улучшения work/life balance. Традиционно многие люди видят в удаленной работе возможность улучшить этот баланс, больше времени проводить с родными и контролировать график. 



Нынешний год с его пандемией коронавируса серьезно изменил многие вещи, и в том числе рынок труда. В новой реальности отношение работодателей к удаленной работе изменилось. Появились и новые инструменты поиска работы, которых не было еще несколько месяцев назад. Все это сегодня позволяет инженерам находить удаленную работу с высокой зарплатой (выше, чем была у них в офисе до начала пандемии), и тратить на это меньше времени, чем раньше. 

Я изучил текущую статистику по рынку труда для разработчиков, материалы, которые публикуют рекрутеры и инженеры, которые сами проводят собеседования, а также поговорил с Александром Жили

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

#### Решение: см. fetch_post_text. Сделано немного не так как по постановке, но более общее решение. Просто разметку распарсить - немного скучно =)

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [64]:
import requests
import json
import datetime as dt
import pandas as pd



def leak_info(email, leak):
    """
    Вспомогательная функция - вытащить в нужном формате инфу по утечке.
    Возвращает словарь. Если ошибка с парсингом - то возвращает None.
    """
    try:
        res = {
            'email': email,
            'date': dt.datetime.fromtimestamp(leak['leak_date']/1000),
            'source': leak['leak_info']['title'],
            'descr': leak['leak_info']['description']
        }
        return res
    except:
        log_exception('Failed to parse leak info: {}'.format(leak))
    return None

def check_email_leaks(email):
    """
    Вернуть список утечек в формате: 
      [{email: str, date: leak-timestamp, source: 'service/leak title', descr: 'leak description'}]
    """
    BASE_URL='https://digibody.avast.com/v1/web/leaks'
    req_params = {'email': email}
    res = requests.post(BASE_URL, data=json.dumps(req_params))
    if res.status_code != 200:
        raise Exception('Bad result status code: {}'.format(res.status_code))
    resp = json.loads(res.text)
    if resp['status'] != 'ok':
        raise Exception('Response status "{}" != "ok"'.format(resp['status']))
        
    res = [ leak_info(email, leak) for leak in resp['value'] ]
    return list(filter(None.__ne__, res))

# проверяем
EMAILS = ['asdf@mail.ru', 'qwer@mail.ru']
leaks = []
for eml in EMAILS:
    try:
        check_res = check_email_leaks(eml)      
        leaks = leaks + check_res
    except:
        log_exception(f'Failed to process email "{eml}"')

df = pd.DataFrame(leaks)
df

,email,date,source,descr
0,asdf@mail.ru,2020-07-16 03:00:00,Ongab,"At an unconfirmed date, the Russian gaming soc..."
1,asdf@mail.ru,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
2,asdf@mail.ru,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
3,asdf@mail.ru,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
4,asdf@mail.ru,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
...,...,...,...,...
128,qwer@mail.ru,2020-05-14 03:00:00,Combo List,This combolist was compiled from a variety of ...
129,qwer@mail.ru,2020-05-14 03:00:00,Combo List,This combolist was compiled from a variety of ...
130,qwer@mail.ru,2019-12-05 03:00:00,Sensitive Source,This source has been marked as sensitive due t...
131,qwer@mail.ru,2019-09-12 03:00:00,Ravnovesie.com,"At an unconfirmed date, Russian multimedia dis..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [66]:
# использовать api напрямую неочень удобно, возьмём либу
%pip install vk

  Created wheel for vk: filename=vk-2.0.2-cp37-none-any.whl size=8276 sha256=64b0dce6034a82f7aaed5d211093a595bb5f815e2601ffd1ea509026a340845e
  Stored in directory: /home/ev42/.cache/pip/wheels/4c/48/d1/09749ec47d9a30d166122773811f4ccb406f5234f2d84fd29d
Successfully built vk
Note: you may need to restart the kernel to use updated packages.


In [70]:
import vk
import pandas as pd

sess  = vk.Session(access_token='')
vkcli = vk.API(sess)

domain = 'netology' 
resp  = vkcli.wall.get(domain=domain, count=50, v='5.122')
posts = resp['items']
df = pd.DataFrame(data={'date': [p['date'] for p in posts], 'text': [p['text'] for p in posts]})
df

,date,text
0,1597402808,Всем привет! Сегодня первый день карьерного кв...
1,1597678680,"Зарядку сделали, перед зеркалом порепетировали..."
2,1597654812,"Как приятно видеть, что работа приносит резуль..."
3,1597650120,Запустили бесплатный курс по основам контекстн...
4,1597580880,"Подборка книг, которые помогут развить математ..."
5,1597566624,Продвижение курса — дело непростое. Но у вас к...
6,1597491720,Личный бренд работает по принципу зачётки: сна...
7,1597480217,"Как успехи у ваших студентов? Поздравляем, вы ..."
8,1597477320,*партнерский пост*\n\nМеждународный конкурс по...
9,1597402812,А вот и первое задание карьерного квеста. Мене...


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
